# Here, I build out the Langevin integrator capacities of my system. 

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import time
import plotly
import plotly.graph_objects as grapher
import nbformat
# from __future__ import annotations
from typing import Optional, Tuple, List, Literal, Callable
from numpy.typing import NDArray
FloatArray = NDArray[np.floating]

from integrator import run
from visualizer import generate_video, generate_plot


In [345]:
# implement our harmonic potentials

def get_harmonic_U(k,L):
    def U(q):
        difs = q[:, None, :] - q[None, :, :]      # Here, we create a matrix of differences q_i - q_j
        difs -= L * np.round(difs / L)             
# The line above solves the Minimum Image Convention Problem. 
# If two particles have a distance that is close to L, we subtract that distance. 
# This basically means that particles choose the closer of two interaction distances: 
# the obvious distance within the torus, or the distance across the toroidal boundary. 
        sumSquares = np.sum(difs**2,axis=2) # sum the x,y, and z components to get an N x N matrix

        return(0.5*k*np.sum(np.triu(sumSquares,k=1))) # we only care about the unique pairs
    return(U) 

def get_harmonic_Nabla(k,L):
    def Nabla(q):
        difs = q[:, None, :] - q[None, :, :]      # See function above for explanatino
        difs -= L * np.round(difs / L)  
        return(k* np.sum(difs,axis = 1)) # sum over j, noting that the dif for j=i is 0
    return(Nabla) 


In [346]:
def apply_A(p,q,dt,M):
    # the A operator doesn't touch p, but it adjusts q (position) via Newton
    q += p/M * dt # careful! numpy will broadcast M.shape = (N,1) but not (N,)... need to keep an eye out
    return(p,q)

def apply_B(p,q,dt,getNablaU):
    # the A operator applies the newtonian force kick 
    F = -getNablaU(q)
    p += F*dt
    return(p,q)

def apply_O(p,q,dt,gamma,M,kB,T,rng):
    # the O operator adds thermal randomness to our system
    # it only adjusts p

    # a = np.exp(-gamma*dt)
    # stdev = np.sqrt(M*kB*T*(1-a**2)) # NOTE: probably want to make this more stable later on
    ## when dt is small, a is close to 1, so 1-a**2 is close to 0; this can make the above unstable
    ## Hence, use the following: 

    a = np.exp(-gamma*dt)
    one_minus_a2 = np.expm1(-2*gamma*dt) * (-1)   # equals 1 - a**2 with better accuracy
    one_minus_a2 = np.clip(one_minus_a2, 0.0, 1.0) # clip to avoid more floating point errors
    stdev = np.sqrt(M*kB*T*(one_minus_a2))
    xi = rng.normal(size = p.shape)
    p = a*p + stdev * xi

    return(p,q)

## Main Function to Run the Simulation

In [347]:
def run(p0:Optional[FloatArray] = None,q0:Optional[FloatArray] = None,N: int = 10,
        M:Optional[FloatArray]=None,
        dt:float = 1.e-3,num_steps:int = 10*3,T:float = 1.0,gamma:float = 1.0,kB:float = 1.0,systemLength:float = 10**4,
        rng = np.random.default_rng(42),
        potential_type = "harmonic",kSpring:float = 1.0,
        timing = False, printing_steps = False):
    
    other_data = {}


    if potential_type == "harmonic":
        getNabla = get_harmonic_Nabla(kSpring,systemLength)
        getU = get_harmonic_U(kSpring,systemLength)

    p=p0
    q=q0


    q_table,p_table,U_table = [q.copy()],[p.copy()],[getU(q)]

    def save_data(p,q):
        q_table.append(q.copy())
        p_table.append(p.copy())
        U_table.append(getU(q))


    def BAOAB(p,q):
        # perform BAOAB
        p,q = apply_B(p,q,dt/2,getNabla)

        p,q = apply_A(p,q,dt/2,M)
        # apply periodic boundary conditions

        q = np.mod(q,systemLength)
        p,q = apply_O(p,q,dt,gamma,M,kB,T,rng)
        p,q = apply_A(p,q,dt/2,M)

        # apply periodic boundary conditions
        q = np.mod(q,systemLength)

        p,q = apply_B(p,q,dt/2,getNabla)
        return(p,q)


    if timing: 
        start = time.perf_counter()

    #### MAIN LOOP ####
    for step in range(num_steps):

        if printing_steps and step>0 and step % int(num_steps//10) == 0:
                print(f"At step {step} out of {num_steps}")

        p,q = BAOAB(p,q)
        save_data(p,q)
        
    
    if timing: 
        end = time.perf_counter()
        other_data['time'] = end-start
    other_data['systemLength'] = systemLength
    other_data['T'] = T
    other_data['kB'] = kB
    other_data['gamma'] = gamma
    other_data['dt'] = dt

    
    return (np.array(p_table),np.array(q_table),np.array(U_table),other_data)


# Experiments

### Basic First Experiment

In [348]:
# First experiment: reduced units

rng = np.random.default_rng(42)


N= 2 # number of particles

p = np.empty((N,3),dtype=float) # momentum vector
q= p.copy() # position vector
M = np.ones((N,1),dtype=float) 
dt = 1.e-3
num_steps = 10**3 
T = 1.0
gamma = 1.0
kB = 10 

# starting positins in a small Gaussian, then center at (0,0,0)
q = 10 * rng.normal(size=(N,3))
q -= q.mean(axis=0, keepdims=True)

# starting momenta are boltzmann distributed with stdev = sqrt(mkBT) per component
p_std = np.sqrt(M * kB * T)         
p0 = 10*p_std * rng.normal(size=(N,3))
# here, we 'center' the momenta so they're not biased in a given direction
p -= p.mean(axis=0, keepdims=True)
p0,q0 = p,q

potential_type = "harmonic"
kSpring=1.0

pTable,qTable,UTable,other_Data = run(p0,q0,num_steps=num_steps,
                           N=N,M=M,dt=dt,gamma=gamma,kB=kB,T=T,
                           potential_type="harmonic",
                           timing = True,printing_steps=True)

print(f"other data = {other_Data}")

print(qTable.shape)

At step 100 out of 1000
At step 200 out of 1000
At step 300 out of 1000
At step 400 out of 1000
At step 500 out of 1000
At step 600 out of 1000
At step 700 out of 1000
At step 800 out of 1000
At step 900 out of 1000
other data = {'time': 0.042986041167750955, 'systemLength': 10000, 'T': 1.0, 'kB': 10, 'gamma': 1.0, 'dt': 0.001}
(1001, 2, 3)


#### Some Simple Timing Experiments: NOT RUNNING
- Note that I reuse most of the parameters from the first experiment

In [349]:
timing_experiments = False
if timing_experiments:
    time_table_in_N = []
    Ns = []
    for exp in np.arange(1,6,0.1):
        N_exp = 10**exp
        print(f"N = {N_exp}")
        Ns.append(exp)
        _,_,_,o = run(p0,q0,num_steps=num_steps,
                            N=N_exp,M=M,dt=dt,gamma=gamma,kB=kB,T=T,
                            potential_type="harmonic",
                            timing = True,printing_steps=False)
        time_table_in_N.append(o['time'])

    plt.plot(Ns,time_table_in_N)
    plt.title('Timing vs N')
    plt.xlabel('N')
    plt.ylabel('Time (s)')
    plt.show()

# Import Visualization Tools

In [2]:
from visualizer import generate_video, generate_plot

def visualize(qTable,L=10**4):
    fig = generate_video(qTable,L)
    fig.show(renderer= "browser")

def plot(qTable,L=10**4):
    fig, ax = generate_plot(qTable,L,step=1,idx=0,unwrap=False)
    plt.show()


# visualize(qTable=qTable)